# Train rt-detr to detect weeds in potato fields

In this notebook we'll train a [rt-detr](https://docs.ultralytics.com/models/rtdetr/) model to detect weeds in a potato fields. 
The target weeds are:
- kikuyo
- lengua de vaca
- diente de león

These weeds are common in potatoes fields. The images was captured in 9-10 meters height with a drone. Then, the images was selected and cropped 
to 1920x1920 pixels. Finally, the images was labeled with Roboflow.

## Install dependencies
First of all, we'll check the GPU, and install the dependencies.

In [ ]:
!python --version
!cat /etc/os-release

In [ ]:
!pip install -q coco-eval pycocotools supervision ultralytics
!pip install -q wandb ipywidgets --upgrade

## Initialize weights & biases
We'll use weights & biases to log the training process.

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

project_name = 'rt-detr-potatoes-final'
name = 'train-default-config'
user_secrets = UserSecretsClient()
key = user_secrets.get_secret('wandb')


wandb.login(key=key)
wandb.init(project=project_name, name=name, job_type='training')

## Create function to create data.yaml file

In [4]:
import yaml

base_config = {
    'path': '/path/to/dataset',
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'names': {
        0: 'class_a',
        1: 'class_b',
        2: 'class_c'
    },
}

default_config = {
    'imgsz': 640,
    'epochs': 100,
    'patience': 100,
    'batch': 8,
    'optimizer': 'auto',
    'split': 'val',
    'save_json': True,
    'iou': 0.7,
    'max_det': 300,
    'lr0': 0.01,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'hsv_h': 0.015,
    'hsv_s': 0.7,
    'hsv_v': 0.4,
    'degrees': 0.0,
    'translate': 0.1,
    'flipud': 0.0,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'erasing': 0.4,
    'crop_fraction': 1.0,
}


def create_configuration_file(data: dict = None, filename: str = 'data.yaml', use_default_config: bool = False):
    '''Create configuration file with default values

    Args:
    ----
        data (dict): Data to write to configuration file
        filename (str): Name of the configuration file
        use_default_config (bool): Create configuration file with default values

    Returns:
    --------
        str: Name of the configuration file
    '''
    data = data or {}

    # Set default values if not provided
    data['path'] = data['path'] if 'path' in data else base_config['path']
    data['train'] = data['train'] if 'train' in data else base_config['train']
    data['val'] = data['val'] if 'val' in data else base_config['val']
    data['test'] = data['test'] if 'test' in data else base_config['test']
    data['names'] = data['names'] if 'names' in data else base_config['names']

    if use_default_config:

        data = {**data, **default_config}

    # Create configuration file
    with open(filename, 'w') as file:
        yaml.dump(data, file, default_flow_style=False)

    return filename

## Create configuration file
We'll create a configuration file with the hyperparameters.

In [3]:
data = {
    'path': '/path/to/dataset',
    'names': {
        0: 'diente_leon',
        1: 'kikuyo',
        2: 'lengua_vaca',
        3: 'otros',
        4: 'papa',
    },
}

create_configuration_file(data)

'data.yaml'

## Prepare model
We'll download the rt-detr model and prepare it before training.

In [ ]:
from ultralytics import RTDETR
from wandb.integration.ultralytics import add_wandb_callback


# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')


# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

## Select the mode that you prefer

In [6]:
from enum import Enum
import ipywidgets as widgets

Mode = Enum('Mode', ['tune', 'train'])

dropdown = widgets.Dropdown(
    options=[Mode.tune.name, Mode.train.name],
    value=Mode.tune.name,
    description='Select one:',
    disabled=False,
)
dropdown

Dropdown(description='Select one:', index=1, options=('train', 'tune'), value='tune')

In [ ]:
mode  = dropdown.value

## Train/Tune the model
Once you have selected the mode, you can train or tune the model.

In [ ]:
if mode == Mode.train.name:
    results = model.train(
        project=project_name, name=name, data='data.yaml', device=[0, 1], batch=8, augment=True
    )
    results = model.val()
else:
    model.tune(
    project=project_name, name=name, data='data.yaml',
    device=[0, 1], batch=8, epochs=40, iterations=35, optimizer='SGD',imgsz=640
)

wandb.finish()

## Save the model
We'll save the model in the output folder.

In [ ]:
model.export()